In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.9 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import kerastuner as kt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
import seaborn as sns
import matplotlib.pyplot as plt

#  Import and read the heart_disease_dataset.csv.
import pandas as pd

url= 'https://raw.githubusercontent.com/Nathanhans/project-4/main/heart_disease_dataset.csv'
heart_disease_df = pd.read_csv(url)
heart_disease_df.head()

<ipython-input-2-15bb455f3564>:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Alabama,Female,Very good,4,0,Within past year (anytime less than 12 months ...,Yes,9,None of them,No,...,1.60,71.67,27.99,No,No,Yes,Yes,"Yes, received Tdap",No,No
1,Alabama,Male,Very good,0,0,Within past year (anytime less than 12 months ...,Yes,6,None of them,No,...,1.78,95.25,30.13,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
2,Alabama,Male,Very good,0,0,Within past year (anytime less than 12 months ...,No,8,"6 or more, but not all",No,...,1.85,108.86,31.66,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
3,Alabama,Female,Fair,5,0,Within past year (anytime less than 12 months ...,Yes,9,None of them,No,...,1.70,90.72,31.32,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
4,Alabama,Female,Good,3,15,Within past year (anytime less than 12 months ...,Yes,5,1 to 5,No,...,1.55,79.38,33.07,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No


In [3]:
#confirming lenght to make sure no data was lost
len(heart_disease_df)

246022

In [4]:
# Drop the non-beneficial columns such as State
heart_disease_df.head()

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Alabama,Female,Very good,4,0,Within past year (anytime less than 12 months ...,Yes,9,None of them,No,...,1.60,71.67,27.99,No,No,Yes,Yes,"Yes, received Tdap",No,No
1,Alabama,Male,Very good,0,0,Within past year (anytime less than 12 months ...,Yes,6,None of them,No,...,1.78,95.25,30.13,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
2,Alabama,Male,Very good,0,0,Within past year (anytime less than 12 months ...,No,8,"6 or more, but not all",No,...,1.85,108.86,31.66,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
3,Alabama,Female,Fair,5,0,Within past year (anytime less than 12 months ...,Yes,9,None of them,No,...,1.70,90.72,31.32,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
4,Alabama,Female,Good,3,15,Within past year (anytime less than 12 months ...,Yes,5,1 to 5,No,...,1.55,79.38,33.07,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No


In [5]:
# nuniuqe to view the unique data in each column
heart_disease_df.nunique()

State                          54
Sex                             2
GeneralHealth                   5
PhysicalHealthDays             31
MentalHealthDays               31
LastCheckupTime                 4
PhysicalActivities              2
SleepHours                     23
RemovedTeeth                    4
HadHeartAttack                  2
HadAngina                       2
HadStroke                       2
HadAsthma                       2
HadSkinCancer                   2
HadCOPD                         2
HadDepressiveDisorder           2
HadKidneyDisease                2
HadArthritis                    2
HadDiabetes                     4
DeafOrHardOfHearing             2
BlindOrVisionDifficulty         2
DifficultyConcentrating         2
DifficultyWalking               2
DifficultyDressingBathing       2
DifficultyErrands               2
SmokerStatus                    4
ECigaretteUsage                 4
ChestScan                       2
RaceEthnicityCategory           5
AgeCategory   

In [6]:
heart_disease_df= heart_disease_df.copy().drop(['State'],axis=1)

In [7]:
target_column = heart_disease_df['HadHeartAttack']
df = heart_disease_df.drop('HadHeartAttack', axis=1)  # Drop the target column from the DataFrame

# Apply get_dummies() to the remaining DataFrame
df_dummies = pd.get_dummies(df)

# Concatenate the target column with the DataFrame of dummy variables
df_processed = pd.concat([target_column, df_dummies], axis=1)


In [8]:
df_processed.head()

,HadHeartAttack,PhysicalHealthDays,MentalHealthDays,SleepHours,HeightInMeters,WeightInKilograms,BMI,Sex_Female,Sex_Male,GeneralHealth_Excellent,...,PneumoVaxEver_Yes,"TetanusLast10Tdap_No, did not receive any tetanus shot in the past 10 years","TetanusLast10Tdap_Yes, received Tdap","TetanusLast10Tdap_Yes, received tetanus shot but not sure what type","TetanusLast10Tdap_Yes, received tetanus shot, but not Tdap",HighRiskLastYear_No,HighRiskLastYear_Yes,CovidPos_No,CovidPos_Tested positive using home test without a health professional,CovidPos_Yes
0,No,4,0,9,1.60,71.67,27.99,1,0,0,...,1,0,1,0,0,1,0,1,0,0
1,No,0,0,6,1.78,95.25,30.13,0,1,0,...,1,0,0,1,0,1,0,1,0,0
2,No,0,0,8,1.85,108.86,31.66,0,1,0,...,1,1,0,0,0,1,0,0,0,1
3,No,5,0,9,1.70,90.72,31.32,1,0,0,...,1,1,0,0,0,1,0,0,0,1
4,No,3,15,5,1.55,79.38,33.07,1,0,0,...,1,1,0,0,0,1,0,1,0,0


In [9]:
# Split our preprocessed data into our features and target arrays
y = df_processed['HadHeartAttack']
X = df_processed.drop(['HadHeartAttack'], axis = 1)


In [ ]:
# Convert y_train to a NumPy array
# y_train = np.array(y_train)

# # Reshape y_train to have one column
# y_train = y_train.reshape(-1, 1)

NameError: name 'y_train' is not defined

In [10]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(184516, 100)
(61506, 100)


In [ ]:
# X_train_scaled_reshaped = np.reshape(X_train_scaled, (-1, 2))
# X_test_scaled_reshaped = np.reshape(X_test_scaled, (-1, 2))

In [ ]:
# print(X_train_scaled_reshaped.shape)
# print(X_test_scaled_reshaped.shape)

(9410316, 2)
(3136806, 2)


In [13]:
# Create instance of labelencoder
le= LabelEncoder()
# Fit the LabelEncoder on the target variable
y_train_encoded = le.fit_transform(y_train)
#Fit the LabelEncoder on the test variable
y_test_encoded= le.fit_transform(y_test)

In [18]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=100))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [19]:
# Import the kerastuner library
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [20]:
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"y_train shape: {y_train_encoded.shape}")
print(f"X_train_scaled data type: {X_train_scaled.dtype}")
print(f"y_train data type: {y_train_encoded.dtype}")

X_train_scaled shape: (184516, 100)
y_train shape: (184516,)
X_train_scaled data type: float64
y_train data type: int64


In [21]:
# Run the kerastuner search for best hyperparameters
# tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))
tuner.search(X_train_scaled,y_train_encoded,epochs=20,validation_data=(X_test_scaled,y_test_encoded))
# tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 24 Complete [00h 05m 31s]
val_accuracy: 0.9478750228881836

Best val_accuracy So Far: 0.9496471881866455
Total elapsed time: 00h 48m 23s

Search: Running Trial #25

Value             |Best Value So Far |Hyperparameter
sigmoid           |relu              |activation
3                 |7                 |first_units
5                 |3                 |num_layers
3                 |3                 |units_0
9                 |1                 |units_1
9                 |9                 |units_2
9                 |7                 |units_3
3                 |7                 |units_4
9                 |1                 |units_5
20                |7                 |tuner/epochs
0                 |3                 |tuner/initial_epoch
0                 |2                 |tuner/bracket
0                 |1                 |tuner/round

Epoch 1/20
5767/5767 [==============================] - 20s 3ms/step - loss: 0.2629 - accuracy: 0.9198 - val_loss: 0.2119 - val_accuracy: 0.

KeyboardInterrupt: 

In [22]:
test = tuner.get_best_hyperparameters(5)

In [27]:
models = tuner.get_best_models(5)[0]

In [28]:
#top hyperparameters
test[0].values

{'activation': 'relu',
 'first_units': 7,
 'num_layers': 3,
 'units_0': 3,
 'units_1': 1,
 'units_2': 9,
 'units_3': 7,
 'units_4': 7,
 'units_5': 1,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 3,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0009'}

In [29]:
models

In [30]:
for v in test:
  print(v.values)

{'activation': 'relu', 'first_units': 7, 'num_layers': 3, 'units_0': 3, 'units_1': 1, 'units_2': 9, 'units_3': 7, 'units_4': 7, 'units_5': 1, 'tuner/epochs': 7, 'tuner/initial_epoch': 3, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0009'}
{'activation': 'relu', 'first_units': 9, 'num_layers': 1, 'units_0': 1, 'units_1': 9, 'units_2': 5, 'units_3': 3, 'units_4': 5, 'units_5': 5, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0016'}
{'activation': 'relu', 'first_units': 9, 'num_layers': 1, 'units_0': 1, 'units_1': 9, 'units_2': 5, 'units_3': 3, 'units_4': 5, 'units_5': 5, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 5, 'num_layers': 1, 'units_0': 5, 'units_1': 1, 'units_2': 3, 'units_3': 9, 'units_4': 5, 'units_5': 5, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0018'}
{'activation': 'relu

In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 5,
 'num_layers': 1,
 'units_0': 7,
 'units_1': 5,
 'units_2': 1,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [31]:
predictions= nn.predict(X_test_scaled)
predictions

NameError: name 'nn' is not defined

In [ ]:
round_array = np.array([int(np.round(i)) for i in predictions])
round_array

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test_encoded, round_array)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_encoded, round_array))

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = models.evaluate(X_test_scaled,y_test_encoded,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1923/1923 - 3s - loss: 0.1439 - accuracy: 0.9496 - 3s/epoch - 1ms/step
Loss: 0.14385299384593964, Accuracy: 0.9496471881866455
